In [66]:
%%oc

MATCH ()-[e:referencedByDoc]->()
DELETE e

In [56]:
%%oc
MATCH (o:OrgKG_Organization)<-[:hasParentCompany*0..]-(o1:OrgKG_Organization)<-[:resolvesToOrg]-(n:ORGANIZATION)<-[]-()<-[:EVENT]-(d:DOCUMENT)
WITH DISTINCT o, d
CREATE (o)-[:referencedByDoc]->(d)

## Jaccard Similarity (common nodes / union of neighbors of either node)
Most of the documents mentioning Amazon do not mention Whole Foods

In [1]:
%%oc
MATCH (o1:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Amazon_(company)"}),(o2:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Whole_Foods_Market"})
CALL neptune.algo.jaccardSimilarity(o1, o2, {edgeLabels: ['referencedByDoc']})
YIELD score
RETURN ID(o1), ID(o2), score

## Overlap Similarity [ common nodes / min(# of neighbors of either node) ]
All of the documents mentioning Whole Foods also mention Amazon

In [2]:
%%oc
MATCH (o1:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Amazon_(company)"}),(o2:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Whole_Foods_Market"})
CALL neptune.algo.overlapSimilarity(o1, o2, {edgeLabels: ['referencedByDoc']})
YIELD score
RETURN ID(o1), ID(o2), score

## We can use Jaccard Similarity across all of the organization references to identify which commonly appear together in the same document

In [3]:
%%oc
MATCH (o1:OrgKG_Organization)-[:referencedByDoc]->(d:DOCUMENT)<-[:referencedByDoc]-(o2:OrgKG_Organization)
CALL neptune.algo.jaccardSimilarity(o1, o2, {edgeLabels: ['referencedByDoc']})
YIELD score
RETURN ID(o1), ID(o2), score, COLLECT(d.title)
ORDER BY score DESC
LIMIT 20

In [4]:
%%oc
MATCH (o:OrgKG_Organization) 
CALL neptune.algo.closenessCentrality(
  o,
  {
    numSources: 1000,
    traversalDirection: "both"
  }
)
YIELD node, score
RETURN ID(node), score
ORDER BY score DESC

In [6]:
%%oc

MATCH p=(n:OrgKG_Organization {`~id`:'resolved_org_Tsawwassen First Nation'})-[e]-(n1)-[e2]-(n2)-[e3]-(n3)
WHERE ID(n1) <> 'http://dbpedia.org/resource/Amazon_(company)' AND ID(n2) <> 'http://dbpedia.org/resource/Amazon_(company)' AND ID(n3) <> 'http://dbpedia.org/resource/Amazon_(company)'
RETURN p
//RETURN DISTINCT TYPE(e), ID(n1), TYPE(e2), ID(n2), COUNT(e2)


In [8]:
%%oc
MATCH (o:OrgKG_Organization) 
CALL neptune.algo.pageRank(
  o,
  {
  }
)
YIELD node, rank
RETURN ID(node), rank
ORDER BY rank DESC

In [29]:
%%oc

MATCH (n)
CALL neptune.algo.scc(
  n,
  {
  }
)
YIELD node, component
WITH component, collect(node) as nodes_in_component
ORDER BY SIZE(nodes_in_component) DESC
RETURN component, SIZE(nodes_in_component) as cnt, nodes_in_component
%LIMIT 10

In [30]:
%%oc

MATCH (n)
CALL neptune.algo.labelPropagation(
n,
  {
  }
)
Yield node, community
WITH community, collect(node) as nodes_in_component
ORDER BY SIZE(nodes_in_component) DESC
RETURN community, SIZE(nodes_in_component) as cnt, nodes_in_component
LIMIT 10

## EXPLORATION QUERIES

In [26]:
%%bash

awscurl -X POST "https://g-fcowfewla8.us-west-2.neptune-graph.amazonaws.com/queries" \
-H "Content-Type: application/x-www-form-urlencoded" \
--region us-west-2 \
--service neptune-graph \
-d "query=MATCH p=(n:OrgKG_Organization)-[e]-(n1)-[e2]-(n2)-[e3]-(n3) WHERE ID(n) = 'resolved_org_Tsawwassen First Nation' AND ID(n1) <> 'http://dbpedia.org/resource/Amazon_(company)' AND ID(n2) <> 'http://dbpedia.org/resource/Amazon_(company)' AND ID(n3) <> 'http://dbpedia.org/resource/Amazon_(company)' RETURN p;&nativeSerialization=false"

{
  "results" : [ {
    "p" : [ {
      "~id" : "resolved_org_Tsawwassen First Nation",
      "~entityType" : "node",
      "~labels" : [ "OrgKG_Organization" ],
      "~properties" : {
        "name" : "Tsawwassen First Nation"
      }
    }, {
      "~id" : "neptune_reserved_1_1152921551851945990",
      "~entityType" : "relationship",
      "~start" : "node__organization_tsawwassen_first_nation",
      "~end" : "resolved_org_Tsawwassen First Nation",
      "~type" : "resolvesToOrg",
      "~properties" : { }
    }, {
      "~id" : "node__organization_tsawwassen_first_nation",
      "~entityType" : "node",
      "~labels" : [ "ORGANIZATION" ],
      "~properties" : {
        "names" : "Tsawwassen First Nation",
        "primaryName" : "Tsawwassen First Nation"
      }
    }, {
      "~id" : "neptune_reserved_1_1152921508902666247",
      "~entityType" : "relationship",
      "~start" : "node__event_e5184a80-8e37-43e0-b1d4-be3e59171178_investment_general_investments3981",
      "~end"

In [4]:
%%oc 

MATCH (n)
RETURN DISTINCT LABELS(n), COUNT(n)
ORDER BY COUNT(n) DESC

In [7]:
%%oc
MATCH (o:OrgKG_Organization)-[e]->(n)
RETURN DISTINCT TYPE(e), LABELS(n)

In [8]:
%%oc
MATCH (o:OrgKG_Organization)<-[e]-(n)
RETURN DISTINCT TYPE(e), LABELS(n)

In [15]:
%%oc
MATCH (d:DOCUMENT)
RETURN d
LIMIT 5

In [39]:
%%oc
MATCH (o:OrgKG_Organization)<-[:hasParentCompany*0..]-(o1:OrgKG_Organization)<-[:resolvesToOrg]-(n:ORGANIZATION)<-[e1]-(n1)<-[e2:EVENT]-(d:DOCUMENT)
RETURN DISTINCT TYPE(e1), LABELS(n1), TYPE(e2), COUNT(d)


## END OF EXPLORATION QUERIES

## OBSOLETE QUERIES

In [40]:
%%oc
MATCH (o:OrgKG_Organization)<-[:hasParentCompany*0..]-(o1:OrgKG_Organization)<-[:resolvesToOrg]-(n:ORGANIZATION)<-[]-()<-[:EVENT]-(d:DOCUMENT)
WITH DISTINCT o, d
CREATE (o)<-[:referencesOrgKG]-(d)


In [22]:
%%oc
MATCH (o:OrgKG_Organization)<-[:referencesOrgKG]-(d:DOCUMENT)
RETURN ID(o), COUNT(d)
ORDER BY COUNT(d) DESC

In [27]:
%%oc
MATCH (o1:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Amazon_(company)"}),(o2:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Whole_Foods_Market"})
CALL neptune.algo.jaccardSimilarity(o1, o2, {edgeLabels: ['referencesOrgKG']})
YIELD score
RETURN ID(o1), ID(o2), score

In [55]:
%%oc
MATCH (o1:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Amazon_(company)"}),(o2:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Whole_Foods_Market"})
CALL neptune.algo.neighbors.common(o1, o2, {edgeLabels: ['referencesOrgKG']})
YIELD common
RETURN common


In [31]:
%%oc
MATCH (o2 {`~id`:"http://dbpedia.org/resource/Whole_Foods_Market"})<-[:referencesOrgKG]-(n)
RETURN COUNT(n)

In [45]:
%%oc --store-to overlap
MATCH (o1:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Amazon_(company)"}),(o2:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Whole_Foods_Market"})
MATCH (o1)<-[:referencesOrgKG]-(n)-[:referencesOrgKG]->(o2)
RETURN COUNT(DISTINCT n) as cnt


In [46]:
overlap["results"][0]["cnt"]

3

In [47]:
%%oc --store-to amazon_only
MATCH (o1:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Amazon_(company)"}),(o2:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Whole_Foods_Market"})
MATCH (o1)<-[:referencesOrgKG]-(n)
RETURN COUNT(DISTINCT n) as cnt

In [48]:
amazon_only["results"][0]["cnt"]

97

In [49]:
%%oc --store-to whole_foods_only
MATCH (o1:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Amazon_(company)"}),(o2:OrgKG_Organization {`~id`:"http://dbpedia.org/resource/Whole_Foods_Market"})
MATCH (o2)<-[:referencesOrgKG]-(n)
RETURN COUNT(DISTINCT n) as cnt

In [50]:
common_neighbors = overlap["results"][0]["cnt"]
amazon_total_neighbors = amazon_only["results"][0]["cnt"]
whole_foods_total_neighbors = whole_foods_only["results"][0]["cnt"]

amazon_similarity = float(common_neighbors) / float(amazon_total_neighbors)
whole_foods_similarity = float(common_neighbors) / float(whole_foods_total_neighbors)
print(amazon_similarity)
print(whole_foods_similarity)

0.030927835051546393
1.0


## END OBSOLETE QUERIES